In [29]:
import numpy as np
import pandas as pd
import os
import sys
import time

In [48]:
import torch
import transformers # huggingface
# pretrained weights = 'bert-base-uncased'
from transformers import BertForSequenceClassification, BertTokenizer, BertModel

In [49]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Wiki dataset

In [237]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

I0110 17:14:37.334326 140734822974912 tokenization_utils.py:398] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /Users/johnhallman/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [283]:
comment_df = pd.read_csv("../data/attack_annotated_comments.tsv", sep ='\t')
annotation_df = pd.read_csv("../data/attack_annotations.tsv",  sep='\t')

In [284]:
cutoff, max_length = 0.2, 1000

In [285]:
comment_df = comment_df.drop(columns=['logged_in', 'ns', 'sample'])
comment_df["comment"] = comment_df["comment"].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
comment_df["comment"] = comment_df["comment"].apply(lambda x: x.replace("TAB_TOKEN", " "))

annotation_df = (annotation_df.groupby("rev_id")["attack"].mean() > cutoff)
annotation_df = annotation_df.to_frame().reset_index()

final_df = pd.merge(comment_df, annotation_df, how='inner', on=['rev_id'])
f = lambda s: np.array(tokenizer.encode(s, pad_to_max_length=True, max_length=max_length))

In [286]:
datasets = {}
for split in final_df['split'].unique(): # train, dev, test
    final_split = final_df[final_df['split'] == split]
    x = final_split['comment'].apply(f).values # tokenize!
    y = final_split['attack'].astype(int).values
    datasets[split] = CustomDataset(x, y)

NameError: name 'CustomDataset' is not defined

In [ ]:
datasets.keys()

# Load fake news dataset

In [318]:
# fake news dataset
body_df = pd.read_csv("../data/fake_news_bodies.csv")
stance_df = pd.read_csv("../data/fake_news_stances.csv")

CPU times: user 85.4 ms, sys: 15 ms, total: 100 ms
Wall time: 103 ms


In [310]:
stance_df.head(2)

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree


In [311]:
body_df.head(2)

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...


In [319]:
%%time
idx_to_id = {body_id:i for (i, body_id) in enumerate(body_df['Body ID'])}
stance_to_idx = {stance: i for i, stance in enumerate(stance_df["Stance"].unique())}
separator = ' ' + tokenizer.sep_token + ' '

x_list = []
y_list = []
f = lambda s: np.array(tokenizer.encode(s, pad_to_max_length=True, max_length=1000))
for body_id, headline, stance in zip(stance_df["Body ID"], stance_df["Headline"], stance_df["Stance"]):
    body = body_df.iloc[idx_to_id[body_id]]['articleBody']
    text = headline + separator + body
    x_list.append(f(text))
    y_list.append(stance_to_idx[stance])

x = np.array(x_list)
y = np.array(y_list)

CPU times: user 9min 33s, sys: 865 ms, total: 9min 34s
Wall time: 9min 34s


In [321]:
x.shape

(49972, 1000)

In [329]:
!pip show transformers

Name: transformers
Version: 2.2.1
Summary: State-of-the-art Natural Language Processing for TensorFlow 2.0 and PyTorch
Home-page: https://github.com/huggingface/transformers
Author: Thomas Wolf, Lysandre Debut, Victor Sanh, Julien Chaumond, Google AI Language Team Authors, Open AI team Authors, Facebook AI Authors, Carnegie Mellon University Authors
Author-email: thomas@huggingface.co
License: Apache
Location: /anaconda/envs/robot-env/lib/python3.6/site-packages
Requires: sacremoses, regex, requests, boto3, numpy, tqdm, sentencepiece
Required-by: 


In [330]:
tokenizer.encode("hello world!", pad_to_max_length=True, max_length=20)

[101, 7592, 2088, 999, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]